In [1]:
import numpy as np
import numpy.linalg as la
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import signal
import os
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
save_path = 'D:\EECE499\Features\EEGFeatures.xlsx'
#save_path = '..\..\EEGFeatures.xlsx'

In [4]:
sample_rate = 32

In [5]:
user_id = 1
clip_id = 1

data_path = './../ASCERTAIN_Raw/EEGData/Movie_P' + str(user_id).zfill(2) + '/EEG_Clip' + str(clip_id) + '.mat'

In [6]:
valid_user_clips = {
    '1' : list(range(1, 37)),
    '2' : list(range(1, 37)),
    '3' : list(range(1, 37)),
    '4' : list(range(1, 37)),
    '5' : list(range(1, 37)),
    '6' : list(range(1, 37)),
    '7' : list(range(1, 37)),
    '8' : list(range(1, 37)),
    '9' : list(range(1, 37)),
    '10' : list(range(1, 37)),
    '11' : list(range(1, 37)),
    '12' : list(range(1, 37)),
    '13' : list(range(1, 37)),
    '14' : list(range(1, 37)),
    '15' : list(range(1, 37)),
    '16' : list(range(1, 37)),
    '17' : list(range(1, 37)),
    '18' : list(range(1, 37)),
    '19' : list(range(1, 37)),
    '20' : list(range(1, 37)),
    '21' : list(range(1, 37)),
    '22' : list(range(1, 37)),
    '23' : list(range(1, 37)),
    '24' : list(range(1, 37)),
    '25' : list(range(1, 37)),
    '26' : list(range(1, 37)),
    '27' : list(range(1, 37)),
    '28' : list(range(1, 37)),
    '29' : list(range(1, 37)),
    '30' : list(range(1, 37)),
    '31' : list(range(1, 37)),
    '32' : list(range(1, 37)),
    '33' : list(range(1, 37)),
    '34' : list(range(1, 37)),
    '35' : list(range(1, 37)),
    '36' : list(range(1, 37)),
    '37' : list(range(1, 37)),
    '38' : list(range(1, 37)),
    '39' : list(range(1, 37)),
    '40' : list(range(1, 37)),
    '41' : list(range(1, 37)),
    '42' : list(range(1, 37)),
    '43' : list(range(1, 37)),
    '44' : list(range(1, 37)),
    '45' : list(range(1, 37)),
    '46' : list(range(1, 37)),
    '47' : list(range(1, 37)),
    '48' : list(range(1, 37)),
    '49' : list(range(1, 37)),
    '50' : list(range(1, 37)),
    '51' : list(range(1, 37)),
    '52' : list(range(1, 37)),
    '53' : list(range(1, 37)),
    '54' : list(range(1, 37)),
    '55' : list(range(1, 37)),
    '56' : list(range(1, 37)),
    '57' : list(range(1, 37)),
    '58' : list(range(1, 37))
}

#valid_user_clips['3'].remove(9)
#valid_user_clips['3'].remove(12)
#valid_user_clips['3'].remove(18)
#valid_user_clips['3'].remove(28)
#valid_user_clips['4'].remove(36)
#valid_user_clips['5'].remove(4)
#valid_user_clips['5'].remove(36)
#valid_user_clips['7'].remove(4)
#valid_user_clips['7'].remove(6)
#valid_user_clips['7'].remove(7)
#valid_user_clips['7'].remove(10)
#valid_user_clips['7'].remove(15)
#valid_user_clips['7'].remove(20)
#valid_user_clips['7'].remove(22)
#valid_user_clips['7'].remove(27)
#valid_user_clips['7'].remove(31)
#valid_user_clips['7'].remove(32)
#valid_user_clips['7'].remove(35)
#valid_user_clips['15'].remove(5)
#valid_user_clips['15'].remove(10)
#valid_user_clips['15'].remove(25)
#valid_user_clips['15'].remove(30)
#valid_user_clips['15'].remove(35)
#valid_user_clips['16'].remove(8)
#valid_user_clips['16'].remove(10)
#valid_user_clips['16'].remove(22)
#valid_user_clips['17'].remove(6)
#valid_user_clips['17'].remove(11)
#valid_user_clips['19'].remove(3)
#valid_user_clips['22'].remove(5)
#valid_user_clips['22'].remove(10)
#valid_user_clips['22'].remove(31)
#valid_user_clips['26'].remove(12)
#valid_user_clips['29'].remove(9)
#valid_user_clips['33'].remove(8)
#valid_user_clips['33'].remove(22)
#valid_user_clips['34'].remove(8)
#valid_user_clips['34'].remove(21)
#valid_user_clips['37'].remove(16)
#valid_user_clips['37'].remove(19)
#valid_user_clips['37'].remove(23)
#valid_user_clips['37'].remove(26)
#valid_user_clips['39'].remove(28)
#valid_user_clips['40'].remove(3)
#valid_user_clips['41'].remove(7)
#valid_user_clips['43'].remove(4)
#valid_user_clips['43'].remove(13)
#valid_user_clips['43'].remove(14)
#valid_user_clips['45'].remove(7)
#valid_user_clips['45'].remove(15)

In [7]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))

    neuro_sky_EEG = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    attention = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    meditation = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    alpha = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    beta = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    delta = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    gamma = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    theta = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    
    presentation_id = []
    
    for clip_id in valid_user_clips[str(user_id)]:
        #print('Clip ID: ' + str(clip_id))
        
        data_path = './../ASCERTAIN_Raw/EEGData/Movie_P' + str(user_id).zfill(2) + '/EEG_Clip' + str(clip_id) + '.mat'
        
        presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        signal = scipy.io.loadmat(data_path)
        neuro_sky_EEG_raw = np.array(signal['ThisEEG'][0])
        attention_raw = np.array(signal['ThisEEG'][0])
        meditation_raw = np.array(signal['ThisEEG'][0])
        alpha_raw = np.array(signal['ThisEEG'][0])
        beta_raw = np.array(signal['ThisEEG'][0])
        delta_raw = np.array(signal['ThisEEG'][0])
        gamma_raw = np.array(signal['ThisEEG'][0])
        theta_raw = np.array(signal['ThisEEG'][0])
        
        #neuro_sky_EEG_raw /= la.norm(neuro_sky_EEG_raw)
        #attention_raw /= la.norm(attention_raw)
        #meditation_raw /= la.norm(meditation_raw)
        #alpha_raw /= la.norm(alpha_raw)
        #beta_raw /= la.norm(beta_raw)
        #delta_raw /= la.norm(delta_raw)
        #gamma_raw /= la.norm(gamma_raw)
        #theta_raw /= la.norm(theta_raw)
        #
        #neuro_sky_EEG_raw = np.abs(neuro_sky_EEG_raw)
        #attention_raw = np.abs(attention_raw)
        #meditation_raw = np.abs(meditation_raw)
        #alpha_raw = np.abs(alpha_raw)
        #beta_raw = np.abs(beta_raw)
        #delta_raw = np.abs(delta_raw)
        #gamma_raw = np.abs(gamma_raw)
        #theta_raw = np.abs(theta_raw)
        
        #plt.plot(neuro_sky_EEG_raw, color = 'blue')
        window = 51 #101
        order = 4 #3
        try:
            neuro_sky_EEG_raw = savgol_filter(neuro_sky_EEG_raw, window, order)
            attention_raw = savgol_filter(attention_raw, window, order)
            meditation_raw = savgol_filter(meditation_raw, window, order)
            alpha_raw = savgol_filter(alpha_raw, window, order)
            beta_raw = savgol_filter(beta_raw, window, order)
            delta_raw = savgol_filter(delta_raw, window, order)
            gamma_raw = savgol_filter(gamma_raw, window, order)
            theta_raw = savgol_filter(theta_raw, window, order)
        except:
            l = len(neuro_sky_EEG_raw)
            neuro_sky_EEG_raw = np.zeros(l)
            attention_raw = np.zeros(l)
            meditation_raw = np.zeros(l)
            alpha_raw = np.zeros(l)
            beta_raw = np.zeros(l)
            delta_raw = np.zeros(l)
            gamma_raw = np.zeros(l)
            theta_raw = np.zeros(l)
        #plt.plot(neuro_sky_EEG_raw, color = 'red')
        #plt.show()

        if os.path.isfile(data_path):
            
            #plt.plot(attention_raw)
            #plt.show()
            #print(neuro_sky_EEG_raw)
            
            # average of the first derivative
            neuro_sky_EEG['avg_first_deriv'].append(np.mean(np.diff(neuro_sky_EEG_raw) / np.diff(range(neuro_sky_EEG_raw.size))))
            attention['avg_first_deriv'].append(np.mean(np.diff(attention_raw) / np.diff(range(attention_raw.size))))
            meditation['avg_first_deriv'].append(np.mean(np.diff(meditation_raw) / np.diff(range(meditation_raw.size))))
            alpha['avg_first_deriv'].append(np.mean(np.diff(alpha_raw) / np.diff(range(alpha_raw.size))))
            beta['avg_first_deriv'].append(np.mean(np.diff(beta_raw) / np.diff(range(beta_raw.size))))
            delta['avg_first_deriv'].append(np.mean(np.diff(delta_raw) / np.diff(range(delta_raw.size))))
            gamma['avg_first_deriv'].append(np.mean(np.diff(gamma_raw) / np.diff(range(gamma_raw.size))))
            theta['avg_first_deriv'].append(np.mean(np.diff(theta_raw) / np.diff(range(theta_raw.size))))
            #print(neuro_sky_EEG)
            
            # percentage of negative differential samples
            deriv = np.diff(neuro_sky_EEG_raw) / np.diff(range(neuro_sky_EEG_raw.size))
            neuro_sky_EEG['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(attention_raw) / np.diff(range(attention_raw.size))
            attention['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(meditation_raw) / np.diff(range(meditation_raw.size))
            meditation['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(alpha_raw) / np.diff(range(alpha_raw.size))
            alpha['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(beta_raw) / np.diff(range(beta_raw.size))
            beta['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(delta_raw) / np.diff(range(delta_raw.size))
            delta['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(gamma_raw) / np.diff(range(gamma_raw.size))
            gamma['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(theta_raw) / np.diff(range(theta_raw.size))
            theta['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            
            # mean number of peaks
            peaks, _ = find_peaks(neuro_sky_EEG_raw)
            neuro_sky_EEG['mean_peaks'].append(len(peaks) / len(neuro_sky_EEG_raw))
            peaks, _ = find_peaks(attention_raw)
            attention['mean_peaks'].append(len(peaks) / len(attention_raw))
            peaks, _ = find_peaks(meditation_raw)
            meditation['mean_peaks'].append(len(peaks) / len(meditation_raw))
            peaks, _ = find_peaks(alpha_raw)
            alpha['mean_peaks'].append(len(peaks) / len(alpha_raw))
            peaks, _ = find_peaks(beta_raw)
            beta['mean_peaks'].append(len(peaks) / len(beta_raw))
            peaks, _ = find_peaks(delta_raw)
            delta['mean_peaks'].append(len(peaks) / len(delta_raw))
            peaks, _ = find_peaks(gamma_raw)
            gamma['mean_peaks'].append(len(peaks) / len(gamma_raw))
            peaks, _ = find_peaks(theta_raw)
            theta['mean_peaks'].append(len(peaks) / len(theta_raw))
            #print(neuro_sky_EEG['mean_peaks'])
            
            # average of first derivative of inverse signal
            neuro_sky_EEG['avg_first_deriv_inv'].append(np.mean(np.diff(1 / neuro_sky_EEG_raw) / np.diff(range(neuro_sky_EEG_raw.size))))
            attention['avg_first_deriv_inv'].append(np.mean(np.diff(1 / attention_raw) / np.diff(range(attention_raw.size))))
            meditation['avg_first_deriv_inv'].append(np.mean(np.diff(1 / meditation_raw) / np.diff(range(meditation_raw.size))))
            alpha['avg_first_deriv_inv'].append(np.mean(np.diff(1 / alpha_raw) / np.diff(range(alpha_raw.size))))
            beta['avg_first_deriv_inv'].append(np.mean(np.diff(1 / beta_raw) / np.diff(range(beta_raw.size))))
            delta['avg_first_deriv_inv'].append(np.mean(np.diff(1 / delta_raw) / np.diff(range(delta_raw.size))))
            gamma['avg_first_deriv_inv'].append(np.mean(np.diff(1 / gamma_raw) / np.diff(range(gamma_raw.size))))
            theta['avg_first_deriv_inv'].append(np.mean(np.diff(1 / theta_raw) / np.diff(range(theta_raw.size))))
            
            # mean number of peaks in inverse signal
            peaks, _ = find_peaks(1 / neuro_sky_EEG_raw)
            neuro_sky_EEG['mean_peaks_inv'].append(len(peaks) / len(neuro_sky_EEG_raw))
            peaks, _ = find_peaks(1 / attention_raw)
            attention['mean_peaks_inv'].append(len(peaks) / len(attention_raw))
            peaks, _ = find_peaks(1 / meditation_raw)
            meditation['mean_peaks_inv'].append(len(peaks) / len(meditation_raw))
            peaks, _ = find_peaks(1 / alpha_raw)
            alpha['mean_peaks_inv'].append(len(peaks) / len(alpha_raw))
            peaks, _ = find_peaks(1 / beta_raw)
            beta['mean_peaks_inv'].append(len(peaks) / len(beta_raw))
            peaks, _ = find_peaks(1 / delta_raw)
            delta['mean_peaks_inv'].append(len(peaks) / len(delta_raw))
            peaks, _ = find_peaks(1 / gamma_raw)
            gamma['mean_peaks_inv'].append(len(peaks) / len(gamma_raw))
            peaks, _ = find_peaks(1 / theta_raw)
            theta['mean_peaks_inv'].append(len(peaks) / len(theta_raw))
            
            # mean of signal
            neuro_sky_EEG['mean'].append(np.mean(neuro_sky_EEG_raw))
            attention['mean'].append(np.mean(attention_raw))
            meditation['mean'].append(np.mean(meditation_raw))
            alpha['mean'].append(np.mean(alpha_raw))
            beta['mean'].append(np.mean(beta_raw))
            delta['mean'].append(np.mean(delta_raw))
            gamma['mean'].append(np.mean(gamma_raw))
            theta['mean'].append(np.mean(theta_raw))
            
            # standard deviation of signal
            neuro_sky_EEG['std'].append(np.std(neuro_sky_EEG_raw))
            attention['std'].append(np.std(attention_raw))
            meditation['std'].append(np.std(meditation_raw))
            alpha['std'].append(np.std(alpha_raw))
            beta['std'].append(np.std(beta_raw))
            delta['std'].append(np.std(delta_raw))
            gamma['std'].append(np.std(gamma_raw))
            theta['std'].append(np.std(theta_raw))
            
            # skewness of signal
            neuro_sky_EEG['skewness'].append(skew(neuro_sky_EEG_raw))
            attention['skewness'].append(skew(attention_raw))
            meditation['skewness'].append(skew(meditation_raw))
            alpha['skewness'].append(skew(alpha_raw))
            beta['skewness'].append(skew(beta_raw))
            delta['skewness'].append(skew(delta_raw))
            gamma['skewness'].append(skew(gamma_raw))
            theta['skewness'].append(skew(theta_raw))
            
            # kurtosis of signal
            neuro_sky_EEG['kurtosis'].append(kurtosis(neuro_sky_EEG_raw))
            attention['kurtosis'].append(kurtosis(attention_raw))
            meditation['kurtosis'].append(kurtosis(meditation_raw))
            alpha['kurtosis'].append(kurtosis(alpha_raw))
            beta['kurtosis'].append(kurtosis(beta_raw))
            delta['kurtosis'].append(kurtosis(delta_raw))
            gamma['kurtosis'].append(kurtosis(gamma_raw))
            theta['kurtosis'].append(kurtosis(theta_raw))
            
            # percent of samples above the mean + std
            neuro_sky_EEG['percent_over_std'].append(len(neuro_sky_EEG_raw[neuro_sky_EEG_raw > np.mean(neuro_sky_EEG_raw) + np.std(neuro_sky_EEG_raw)]) / len(neuro_sky_EEG_raw))
            attention['percent_over_std'].append(len(attention_raw[attention_raw > np.mean(attention_raw) + np.std(attention_raw)]) / len(attention_raw))
            meditation['percent_over_std'].append(len(meditation_raw[meditation_raw > np.mean(meditation_raw) + np.std(meditation_raw)]) / len(meditation_raw))
            alpha['percent_over_std'].append(len(alpha_raw[alpha_raw > np.mean(alpha_raw) + np.std(alpha_raw)]) / len(alpha_raw))
            beta['percent_over_std'].append(len(beta_raw[beta_raw > np.mean(beta_raw) + np.std(beta_raw)]) / len(beta_raw))
            delta['percent_over_std'].append(len(delta_raw[delta_raw > np.mean(delta_raw) + np.std(delta_raw)]) / len(delta_raw))
            gamma['percent_over_std'].append(len(gamma_raw[gamma_raw > np.mean(gamma_raw) + np.std(gamma_raw)]) / len(gamma_raw))
            theta['percent_over_std'].append(len(theta_raw[theta_raw > np.mean(theta_raw) + np.std(theta_raw)]) / len(theta_raw))
            
            # percent of samples below mean - std
            neuro_sky_EEG['percent_under_std'].append(len(neuro_sky_EEG_raw[neuro_sky_EEG_raw < np.mean(neuro_sky_EEG_raw) - np.std(neuro_sky_EEG_raw)]) / len(neuro_sky_EEG_raw))
            attention['percent_under_std'].append(len(attention_raw[attention_raw < np.mean(attention_raw) - np.std(attention_raw)]) / len(attention_raw))
            meditation['percent_under_std'].append(len(meditation_raw[meditation_raw < np.mean(meditation_raw) - np.std(meditation_raw)]) / len(meditation_raw))
            alpha['percent_under_std'].append(len(alpha_raw[alpha_raw < np.mean(alpha_raw) - np.std(alpha_raw)]) / len(alpha_raw))
            beta['percent_under_std'].append(len(beta_raw[beta_raw < np.mean(beta_raw) - np.std(beta_raw)]) / len(beta_raw))
            delta['percent_under_std'].append(len(delta_raw[delta_raw < np.mean(delta_raw) - np.std(delta_raw)]) / len(delta_raw))
            gamma['percent_under_std'].append(len(gamma_raw[gamma_raw < np.mean(gamma_raw) - np.std(gamma_raw)]) / len(gamma_raw))
            theta['percent_under_std'].append(len(theta_raw[theta_raw < np.mean(theta_raw) - np.std(theta_raw)]) / len(theta_raw))
        
    #for key, value in neuro_sky_EEG.items():
    #    print('neuro_sky_EEG.' + key + '=' + str(len(value)))
    #for key, value in attention.items():
    #    print('attention.' + key + '=' + str(len(value)))
    #for key, value in meditation.items():
    #    print('meditation.' + key + '=' + str(len(value)))
    #for key, value in alpha.items():
    #    print('alpha.' + key + '=' + str(len(value)))
    #for key, value in beta.items():
    #    print('beta.' + key + '=' + str(len(value)))
    #for key, value in delta.items():
    #    print('delta.' + key + '=' + str(len(value)))
    #for key, value in gamma.items():
    #    print('gamma.' + key + '=' + str(len(value)))
    #for key, value in theta.items():
    #    print('theta.' + key + '=' + str(len(value)))

    # saving all the data into dataframe
    user_data = pd.DataFrame({
        'presentation_id' : presentation_id,
        'neur_sky_avg_first_deriv' : neuro_sky_EEG['avg_first_deriv'],
        'neur_sky_percent_neg_deriv' : neuro_sky_EEG['percent_neg_deriv'],
        'neur_sky_mean_peaks' : neuro_sky_EEG['mean_peaks'],
        'neur_sky_avg_first_deriv_inv' : neuro_sky_EEG['avg_first_deriv_inv'],
        'neur_sky_mean_peaks_inv' : neuro_sky_EEG['mean_peaks_inv'],
        'neur_sky_mean' : neuro_sky_EEG['mean'],
        'neur_sky_std' : neuro_sky_EEG['std'],
        'neur_sky_skewness' : neuro_sky_EEG['skewness'],
        'neur_sky_kurtosis' : neuro_sky_EEG['kurtosis'],
        'neur_sky_percent_over_std' : neuro_sky_EEG['percent_over_std'],
        'neur_sky_percent_under_std' : neuro_sky_EEG['percent_under_std'],
        'attention_avg_first_deriv' : attention['avg_first_deriv'],
        'attention_percent_neg_deriv' : attention['percent_neg_deriv'],
        'attention_mean_peaks' : attention['mean_peaks'],
        'attention_avg_first_deriv_inv' : attention['avg_first_deriv_inv'],
        'attention_mean_peaks_inv' : attention['mean_peaks_inv'],
        'attention_mean' : attention['mean'],
        'attention_std' : attention['std'],
        'attention_skewness' : attention['skewness'],
        'attention_kurtosis' : attention['kurtosis'],
        'attention_percent_over_std' : attention['percent_over_std'],
        'attention_percent_under_std' : attention['percent_under_std'],
        'meditation_avg_first_deriv' : meditation['avg_first_deriv'],
        'meditation_percent_neg_deriv' : meditation['percent_neg_deriv'],
        'meditation_mean_peaks' : meditation['mean_peaks'],
        'meditation_avg_first_deriv_inv' : meditation['avg_first_deriv_inv'],
        'meditation_mean_peaks_inv' : meditation['mean_peaks_inv'],
        'meditation_mean' : meditation['mean'],
        'meditation_std' : meditation['std'],
        'meditation_skewness' : meditation['skewness'],
        'meditation_kurtosis' : meditation['kurtosis'],
        'meditation_percent_over_std' : meditation['percent_over_std'],
        'meditation_percent_under_std' : meditation['percent_under_std'],
        'alpha_avg_first_deriv' : alpha['avg_first_deriv'],
        'alpha_percent_neg_deriv' : alpha['percent_neg_deriv'],
        'alpha_mean_peaks' : alpha['mean_peaks'],
        'alpha_avg_first_deriv_inv' : alpha['avg_first_deriv_inv'],
        'alpha_mean_peaks_inv' : alpha['mean_peaks_inv'],
        'alpha_mean' : alpha['mean'],
        'alpha_std' : alpha['std'],
        'alpha_skewness' : alpha['skewness'],
        'alpha_kurtosis' : alpha['kurtosis'],
        'alpha_percent_over_std' : alpha['percent_over_std'],
        'alpha_percent_under_std' : alpha['percent_under_std'],
        'beta_avg_first_deriv' : beta['avg_first_deriv'],
        'beta_percent_neg_deriv' : beta['percent_neg_deriv'],
        'beta_mean_peaks' : beta['mean_peaks'],
        'beta_avg_first_deriv_inv' : beta['avg_first_deriv_inv'],
        'beta_mean_peaks_inv' : beta['mean_peaks_inv'],
        'beta_mean' : beta['mean'],
        'beta_std' : beta['std'],
        'beta_skewness' : beta['skewness'],
        'beta_kurtosis' : beta['kurtosis'],
        'beta_percent_over_std' : beta['percent_over_std'],
        'beta_percent_under_std' : beta['percent_under_std'],
        'delta_avg_first_deriv' : delta['avg_first_deriv'],
        'delta_percent_neg_deriv' : delta['percent_neg_deriv'],
        'delta_mean_peaks' : delta['mean_peaks'],
        'delta_avg_first_deriv_inv' : delta['avg_first_deriv_inv'],
        'delta_mean_peaks_inv' : delta['mean_peaks_inv'],
        'delta_mean' : delta['mean'],
        'delta_std' : delta['std'],
        'delta_skewness' : delta['skewness'],
        'delta_kurtosis' : delta['kurtosis'],
        'delta_percent_over_std' : delta['percent_over_std'],
        'delta_percent_under_std' : delta['percent_under_std'],
        'gamma_avg_first_deriv' : gamma['avg_first_deriv'],
        'gamma_percent_neg_deriv' : gamma['percent_neg_deriv'],
        'gamma_mean_peaks' : gamma['mean_peaks'],
        'gamma_avg_first_deriv_inv' : gamma['avg_first_deriv_inv'],
        'gamma_mean_peaks_inv' : gamma['mean_peaks_inv'],
        'gamma_mean' : gamma['mean'],
        'gamma_std' : gamma['std'],
        'gamma_skewness' : gamma['skewness'],
        'gamma_kurtosis' : gamma['kurtosis'],
        'gamma_percent_over_std' : gamma['percent_over_std'],
        'gamma_percent_under_std' : gamma['percent_under_std'],
        'theta_avg_first_deriv' : theta['avg_first_deriv'],
        'theta_percent_neg_deriv' : theta['percent_neg_deriv'],
        'theta_mean_peaks' : theta['mean_peaks'],
        'theta_avg_first_deriv_inv' : theta['avg_first_deriv_inv'],
        'theta_mean_peaks_inv' : theta['mean_peaks_inv'],
        'theta_mean' : theta['mean'],
        'theta_std' : theta['std'],
        'theta_skewness' : theta['skewness'],
        'theta_kurtosis' : theta['kurtosis'],
        'theta_percent_over_std' : theta['percent_over_std']#,
        #'theta_percent_under_std' : theta['percent_under_std']
    })
        
    data_frames.append(user_data)

User ID: 1
User ID: 2
User ID: 3


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may b

User ID: 4


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may b

User ID: 5


...

User ID: 55


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may b

User ID: 56
User ID: 57


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may b

User ID: 58


In [8]:
EEG_Features = pd.concat(data_frames,ignore_index=True)

In [9]:
EEG_Features.to_excel(save_path)

data_path = './../ASCERTAIN_Features/ASCERTAIN_Features/Dt_EEGFeatures.mat'
signal = scipy.io.loadmat(data_path)
print(signal['EEGFeatures_58'][0, 0][:, 11])
print(attention['mean'])

mean = np.array(signal['EEGFeatures_58'][0, 0][:, 0])
plt.plot(neuro_sky_EEG['mean'])
plt.plot(mean, color='red')
plt.show()
std = np.array(signal['EEGFeatures_58'][0, 0][:, 1])
plt.plot(neuro_sky_EEG['std'])
plt.plot(std, color='red')
plt.show()
skew = np.array(signal['EEGFeatures_58'][0, 0][:, 2])
plt.plot(neuro_sky_EEG['skewness'])
plt.plot(skew, color='red')
plt.show()
kurtosis = np.array(signal['EEGFeatures_58'][0, 0][:, 3])
plt.plot(neuro_sky_EEG['kurtosis'])
plt.plot(kurtosis, color='red')
plt.show()

In [10]:
EEG_Features

,presentation_id,neur_sky_avg_first_deriv,neur_sky_percent_neg_deriv,neur_sky_mean_peaks,neur_sky_avg_first_deriv_inv,neur_sky_mean_peaks_inv,neur_sky_mean,neur_sky_std,neur_sky_skewness,neur_sky_kurtosis,...,theta_avg_first_deriv,theta_percent_neg_deriv,theta_mean_peaks,theta_avg_first_deriv_inv,theta_mean_peaks_inv,theta_mean,theta_std,theta_skewness,theta_kurtosis,theta_percent_over_std
0,0101,0.000456,0.497551,0.064891,-5.478872e-09,0.064891,321.454998,42.611445,-0.893317,4.586420,...,0.000456,0.497551,0.064891,-5.478872e-09,0.064891,321.454998,42.611445,-0.893317,4.586420,0.111417
1,0102,-0.041128,0.482517,0.065238,2.943218e-07,0.065704,326.183187,36.961813,0.064582,3.117498,...,-0.041128,0.482517,0.065238,2.943218e-07,0.065704,326.183187,36.961813,0.064582,3.117498,0.116496
2,0103,0.027912,0.498580,0.066402,-2.834131e-07,0.066402,323.939113,20.787423,-0.139194,0.143962,...,0.027912,0.498580,0.066402,-2.834131e-07,0.066402,323.939113,20.787423,-0.139194,0.143962,0.136209
3,0104,-0.015042,0.507161,0.039280,1.604774e-07,0.038913,326.173718,60.787419,0.539297,2.577120,...,-0.015042,0.507161,0.039280,1.604774e-07,0.038913,326.173718,60.787419,0.539297,2.577120,0.103524
4,0105,-0.019043,0.493129,0.076179,2.667501e-07,0.075866,339.677614,64.403706,0.659600,1.534139,...,-0.019043,0.493129,0.076179,2.667501e-07,0.075866,339.677614,64.403706,0.659600,1.534139,0.143615
5,0106,0.023455,0.487688,0.046218,-3.058513e-07,0.045618,326.868829,52.512605,-0.014891,3.387955,...,0.023455,0.487688,0.046218,-3.058513e-07,0.045618,326.868829,52.512605,-0.014891,3.387955,0.107443
6,0107,-0.022242,0.514226,0.052376,2.173288e-07,0.052723,330.132597,47.941760,0.605935,1.784591,...,-0.022242,0.514226,0.052376,2.173288e-07,0.052723,330.132597,47.941760,0.605935,1.784591,0.126951
7,0108,0.031976,0.477855,0.045200,-2.271890e-07,0.045200,328.658176,58.477342,1.026217,4.082785,...,0.031976,0.477855,0.045200,-2.271890e-07,0.045200,328.658176,58.477342,1.026217,4.082785,0.109972
8,0109,-0.037174,0.479715,0.070410,5.079303e-07,0.070410,321.741908,28.220718,0.039848,6.804236,...,-0.037174,0.479715,0.070410,5.079303e-07,0.070410,321.741908,28.220718,0.039848,6.804236,0.098485
9,0110,-0.044136,0.520125,0.035380,4.085041e-07,0.036767,326.156071,86.504644,-0.547994,3.418555,...,-0.044136,0.520125,0.035380,4.085041e-07,0.036767,326.156071,86.504644,-0.547994,3.418555,0.114117
